In [ ]:
import pickle
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from loguru import logger as log
import configparser
import uuid

In [ ]:
config = configparser.ConfigParser()
config.read('../config.ini')

In [ ]:
qdrant_client = QdrantClient(url=config['production']['QdrantUrl'])

qdrant_client.recreate_collection(
        collection_name=config['production']['CollectionName'],
        vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    )

In [ ]:
with open("../../data/data.pkl", "rb") as f:
    products = pickle.load(f)

In [ ]:
for product in products:
    points = [
            PointStruct(
                id=uuid.uuid4().__str__(), 
                vector=embedding.tolist(),
                payload={**product[0]},
            )
            for embedding in product[1]
        ]
    
    qdrant_client.upsert(collection_name=config['production']['CollectionName'], points=points)
    log.info('embedding of product {} added to qdrant successfully',str(product[0]['id']))